In [1]:
import sys
import os

current_directory = os.getcwd()        
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)


from preprocess.GraphTransformerPrerocess import graph_water_transformer_cov_process_for_gate_predictor
from preprocess.graph import graph_topology, graph_topology_5
from preprocess.BaselinePrerocess import gcn_process_for_gate_predictor
from baselines.gcn import gcn_gate_generator_1

from models.graph_water_transformer_cov import graph_water_transformer_cov_gate_predictor

from losses.loss import gate_loss, water_level_threshold
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from postprocess.threshold import flood_threshold, drought_threshold, flood_threshold_t1, drought_threshold_t1
from spektral.layers import GCNConv

2023-08-12 17:13:09.893090: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 17:13:11.184220: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [3]:
n_hours = 72
K = 24 
masked_value = 1e-10
split_1 = 0.8
split_2 = 0.9
sigma2 = 0.1
epsilon = 0.5

In [4]:
train_X_mask, val_X_mask, test_X_mask, \
train_ws_y, val_ws_y, test_ws_y, \
train_gate_y, val_gate_y, test_gate_y, \
scaler, ws_scaler, gate_scaler = gcn_process_for_gate_predictor(n_hours, K, masked_value, split_1, split_2)

Index(['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
      dtype='object')
train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape, train_gate_y.shape, val_gate_y.shape, test_gate_y.shape: (77069, 14, 96) (9634, 14, 96) (19268, 14, 96) (77069, 96) (9634, 96) (19268, 96) (77069, 24, 5) (9634, 24, 5) (19268, 24, 5)


In [5]:
train_cov, val_cov, test_cov, \
train_tws_reshape, val_tws_reshape, test_tws_reshape, \
train_gate_pump_y, val_gate_pump_y, test_gate_pump_y, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler, gate_scalar = graph_water_transformer_cov_process_for_gate_predictor(n_hours, K, masked_value, split_1, split_2)

Index(['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
      dtype='object')
train_tws/val_tws/test_tws: (77069, 5, 72) (9634, 5, 72) (19268, 5, 72) 
 train_cov/val_cov/test_cov: (77069, 96, 12) (9634, 96, 12) (19268, 96, 12) 
 train_ws_y/val_ws_y/test_ws_y: (77069, 96) (9634, 96) (19268, 96) 
  train_gate_pump_y/val_gate_pump_y/test_gate_pump_y: (77069, 24, 7) (9634, 24, 7) (19268, 24, 7)


### Graph topology

In [6]:
train_adj_mat_gcn, val_adj_mat_gcn, test_adj_mat_gcn = graph_topology(n_hours, K, sigma2, epsilon, len(train_ws_y), len(val_ws_y), len(test_ws_y))

node_indices: [ 0  0  0  0  0  1  1  1  2  2  2  3  3  4  4  4  4  5  5  5  6  6  6  7
  7  7  8  8  8  8  8  9  9  9 10 10 10 11 11 11 12 12 12 12 13 13 13 13
 13 13 13 13 13 13 13 13 13 13] 
neighbor_indices: [ 1  4  8 12 13  1  4 13  3  4 13  2 13  0  1  2 13  7  8 13  7  8 13  5
  6 13  0  5  6 12 13 11 12 13 11 12 13  9 10 13  0  9 10 13  0  1  2  3
  4  5  6  7  8  9 10 11 12 13]
number of nodes: 14, number of edges: 58


In [7]:
train_adj_mat, val_adj_mat, test_adj_mat = graph_topology_5(n_hours, K, sigma2, epsilon, len(train_ws_y), len(val_ws_y), len(test_ws_y))

node_indices: [0 0 0 0 1 1 2 2 3 3 4 4] 
neighbor_indices: [1 2 3 4 0 2 0 1 0 4 0 3]
number of nodes: 5, number of edges: 12


### Model

#### Gate & pump predictor

In [8]:
n_nodes = train_adj_mat_gcn.shape[-1]
n_timesteps = train_X_mask.shape[-1]

In [9]:
# ===== model parameters ======
gcn_unit1 = 32
gcn_unit2 = 16
lstm_units = 64
dropout = 0
gate_min = 0
gate_max = 1


learning_rate = 5e-4
decay_steps = 10000
decay_rate = 0.9
PATIENCE = 100
EPOCHS = 700
BATCH = 512

In [10]:
# n_nodes, n_timesteps, gcn1, gcn2, lstm_unit
gate_predictor, GCNConv = gcn_gate_generator_1(gcn1=gcn_unit1,
                                               gcn2=gcn_unit2,
                                               n_nodes=n_nodes,
                                               n_timesteps=n_timesteps,
                                               lstm_unit=lstm_units,
                                               dropout=dropout,
                                               masked_value=masked_value,
                                               gate_min=gate_min,
                                               gate_max=gate_max
                                              )
gate_predictor._name = "gate_predictor"
# gate_predictor.summary()

2023-08-12 17:13:28.254292: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10402 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:88:00.0, compute capability: 6.1


In [11]:
gate_predictor.compile(optimizer='adam', loss='mse')

#### water stage predictor

In [12]:
ws_predictor = load_model('../saved_models/WaLeF_gtn_p.h5', custom_objects={'GCNConv': GCNConv})

for layer in ws_predictor.layers:
    layer.trainable = False

ws_predictor._name = 'ws_predictor'    
# ws_predictor.summary()

#### Combine gate_predictor and trained ws_predictor

In [13]:
inputs_feat = Input(shape=(14, 96), name='inputs_feat') # 96, 16
inputs_adj_gcn = Input(shape=(14, 14), name='inputs_adj_gcn')
inputs_tws = Input(shape=(5, 72), name='input_tws')
inputs_adj = Input(shape=(5, 5), name='input_adj')

# ================ gate_predictor ================
gate_predictor_output = gate_predictor([inputs_feat, inputs_adj_gcn])  # (24, 7)


# ============  future inputs with replaced gate & pump prediction ============
inputs_ae_reshape = layers.Reshape((96, 14))(inputs_feat)
replaced_future_gate_pump = layers.Concatenate(axis=-1)([inputs_ae_reshape[:, n_hours:, :2], 
                                                         gate_predictor_output, 
                                                         inputs_ae_reshape[:, n_hours:, 9:]
                                                        ]
                                                       )

# ============ original past inputs + future inputs with replaced gate & pump prediction ============
merged_inputs = layers.Concatenate(axis=1)([inputs_ae_reshape[:, :n_hours, :], replaced_future_gate_pump])  # (96, 16)
merged_inputs_cov = merged_inputs[:, :, :12]

ws_predictor_output = ws_predictor([merged_inputs_cov, inputs_tws, inputs_adj])

filda = Model(inputs=[inputs_feat, inputs_adj_gcn, inputs_tws, inputs_adj], outputs=[gate_predictor_output, ws_predictor_output], name='filda')
# filda.summary()

In [20]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
                                                          decay_steps=decay_steps,
                                                          decay_rate=decay_rate)

filda.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss=[gate_loss, water_level_threshold], 
              loss_weights=[0.0, 3.0]
             )


es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=100)
mc = ModelCheckpoint('../saved_models/gcn_gtnp_1.h5'.format(n_hours, K),
                     monitor='val_ws_predictor_loss',
                     mode='min',
                     verbose=2, 
                     custom_objects={'gate_loss':gate_loss, 'water_level_threshold':water_level_threshold}, 
                     save_best_only=True)


history = filda.fit([train_X_mask, train_adj_mat_gcn, train_tws_reshape, train_adj_mat], [train_gate_pump_y, train_ws_y],
                   validation_data=([val_X_mask, val_adj_mat_gcn, val_tws_reshape, val_adj_mat], [val_gate_pump_y, val_ws_y]),
                   batch_size=BATCH, 
                   epochs=EPOCHS, 
                   verbose=2, 
                   callbacks=[es, mc],
                   shuffle=True,
                  )


plt.rcParams["figure.figsize"] = (6, 4)
plt.plot(history.history['ws_predictor_loss'], label='train')
plt.plot(history.history['val_ws_predictor_loss'], label='val')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(fontsize=14)
plt.title("Training loss vs Testing loss", fontsize=18)
# plt.savefig('graph/rnn_loss.png', dpi=300)
plt.show()

Epoch 1/700

Epoch 1: val_ws_predictor_loss improved from inf to 2.82125, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 17s - loss: 0.7819 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.2606 - val_loss: 8.4637 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 2.8212 - 17s/epoch - 114ms/step
Epoch 2/700

Epoch 2: val_ws_predictor_loss improved from 2.82125 to 2.41291, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 9s - loss: 0.7368 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.2456 - val_loss: 7.2387 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 2.4129 - 9s/epoch - 57ms/step
Epoch 3/700

Epoch 3: val_ws_predictor_loss improved from 2.41291 to 2.23439, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 9s - loss: 0.6855 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.2285 - val_loss: 6.7032 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 2.2344 - 9s/epoch - 57ms/step
Epoch 4/700

Epoch 4: v

Epoch 28/700

Epoch 28: val_ws_predictor_loss improved from 1.54986 to 1.54830, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 9s - loss: 0.5908 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1969 - val_loss: 4.6449 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.5483 - 9s/epoch - 57ms/step
Epoch 29/700

Epoch 29: val_ws_predictor_loss improved from 1.54830 to 1.54031, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 9s - loss: 0.5908 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1969 - val_loss: 4.6209 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.5403 - 9s/epoch - 57ms/step
Epoch 30/700

Epoch 30: val_ws_predictor_loss improved from 1.54031 to 1.53971, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 9s - loss: 0.5906 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1969 - val_loss: 4.6191 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.5397 - 9s/epoch - 57ms/step
Epoch 31/700

Ep

151/151 - 8s - loss: 0.5882 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1961 - val_loss: 4.5165 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.5055 - 8s/epoch - 56ms/step
Epoch 55/700

Epoch 55: val_ws_predictor_loss did not improve from 1.50122
151/151 - 8s - loss: 0.5882 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1961 - val_loss: 4.5207 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.5069 - 8s/epoch - 56ms/step
Epoch 56/700

Epoch 56: val_ws_predictor_loss did not improve from 1.50122
151/151 - 8s - loss: 0.5881 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1960 - val_loss: 4.5148 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.5049 - 8s/epoch - 56ms/step
Epoch 57/700

Epoch 57: val_ws_predictor_loss improved from 1.50122 to 1.49874, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 8s - loss: 0.5882 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1961 - val_loss: 4.4962 - val_gat

151/151 - 9s - loss: 0.5871 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1957 - val_loss: 4.4658 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4886 - 9s/epoch - 56ms/step
Epoch 83/700

Epoch 83: val_ws_predictor_loss did not improve from 1.48860
151/151 - 8s - loss: 0.5871 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1957 - val_loss: 4.4697 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4899 - 8s/epoch - 56ms/step
Epoch 84/700

Epoch 84: val_ws_predictor_loss improved from 1.48860 to 1.48848, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 9s - loss: 0.5870 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1957 - val_loss: 4.4654 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4885 - 9s/epoch - 57ms/step
Epoch 85/700

Epoch 85: val_ws_predictor_loss improved from 1.48848 to 1.48616, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 9s - loss: 0.5870 - gate_predictor_loss: 0.0000e+00 - ws_

151/151 - 8s - loss: 0.5864 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1955 - val_loss: 4.4399 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4800 - 8s/epoch - 56ms/step
Epoch 111/700

Epoch 111: val_ws_predictor_loss improved from 1.47918 to 1.47737, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 9s - loss: 0.5864 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1955 - val_loss: 4.4321 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4774 - 9s/epoch - 56ms/step
Epoch 112/700

Epoch 112: val_ws_predictor_loss did not improve from 1.47737
151/151 - 8s - loss: 0.5863 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1954 - val_loss: 4.4406 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4802 - 8s/epoch - 56ms/step
Epoch 113/700

Epoch 113: val_ws_predictor_loss did not improve from 1.47737
151/151 - 8s - loss: 0.5864 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1955 - val_loss: 4.4394 - v

151/151 - 9s - loss: 0.5857 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1952 - val_loss: 4.3825 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4608 - 9s/epoch - 56ms/step
Epoch 139/700

Epoch 139: val_ws_predictor_loss did not improve from 1.46084
151/151 - 9s - loss: 0.5856 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1952 - val_loss: 4.3886 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4629 - 9s/epoch - 56ms/step
Epoch 140/700

Epoch 140: val_ws_predictor_loss did not improve from 1.46084
151/151 - 8s - loss: 0.5856 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1952 - val_loss: 4.3845 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4615 - 8s/epoch - 56ms/step
Epoch 141/700

Epoch 141: val_ws_predictor_loss improved from 1.46084 to 1.46047, saving model to ../saved_models/gcn_gtnp_1.h5
151/151 - 8s - loss: 0.5855 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1952 - val_loss: 4.3814 - v

Epoch 167/700

Epoch 167: val_ws_predictor_loss did not improve from 1.45456
151/151 - 8s - loss: 0.5852 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1951 - val_loss: 4.3876 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4625 - 8s/epoch - 56ms/step
Epoch 168/700

Epoch 168: val_ws_predictor_loss did not improve from 1.45456
151/151 - 8s - loss: 0.5853 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1951 - val_loss: 4.3697 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4566 - 8s/epoch - 56ms/step
Epoch 169/700

Epoch 169: val_ws_predictor_loss did not improve from 1.45456
151/151 - 8s - loss: 0.5852 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1951 - val_loss: 4.3732 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4577 - 8s/epoch - 56ms/step
Epoch 170/700

Epoch 170: val_ws_predictor_loss did not improve from 1.45456
151/151 - 8s - loss: 0.5852 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1

151/151 - 8s - loss: 0.5850 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1950 - val_loss: 4.3715 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4572 - 8s/epoch - 56ms/step
Epoch 197/700

Epoch 197: val_ws_predictor_loss did not improve from 1.45315
151/151 - 8s - loss: 0.5850 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1950 - val_loss: 4.3641 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4547 - 8s/epoch - 56ms/step
Epoch 198/700

Epoch 198: val_ws_predictor_loss did not improve from 1.45315
151/151 - 9s - loss: 0.5849 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1950 - val_loss: 4.3626 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predictor_loss: 1.4542 - 9s/epoch - 56ms/step
Epoch 199/700

Epoch 199: val_ws_predictor_loss did not improve from 1.45315
151/151 - 8s - loss: 0.5849 - gate_predictor_loss: 0.0000e+00 - ws_predictor_loss: 0.1950 - val_loss: 4.3653 - val_gate_predictor_loss: 0.0000e+00 - val_ws_predict

KeyboardInterrupt: 

### Performance

In [16]:

saved_model = load_model('../saved_models/gcn_gtnp_1.h5',
                         custom_objects={'gate_loss':gate_loss, 
                                         'water_level_threshold':water_level_threshold,
                                         'GCNConv': GCNConv
                                        }
                        )


In [17]:
start_time = time.perf_counter()


gate_pump_pred, ws_pred = saved_model.predict([test_X_mask, test_adj_mat_gcn, test_tws_reshape, test_adj_mat])

print(gate_pump_pred.shape)
print(ws_pred.shape)


end_time = time.perf_counter()
used_time = end_time - start_time
print(f"Usded time: {used_time} seconds")

2023-08-12 17:14:40.213486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


603/603 [==============================] - 6s 7ms/step
(19268, 24, 7)
(19268, 96)
Usded time: 7.268814068287611 seconds


#### ws pred, gate pred

In [16]:
ws_pred_gate_pred_inv = ws_scaler.inverse_transform(ws_pred)
ws_pred_gate_pred_inv = ws_pred_gate_pred_inv.reshape((-1, K, 4))
ws_pred_gate_pred_inv.shape

(19268, 24, 4)

#### ws true, gate true

In [17]:
ws_true_gate_true = test_ws_y
ws_true_gate_true_inv = ws_scaler.inverse_transform(ws_true_gate_true)
ws_true_gate_true_inv = ws_true_gate_true_inv.reshape((-1, K, 4))
ws_true_gate_true_inv.shape

(19268, 24, 4)

#### ws pred, gate true

In [18]:
ws_predictor = load_model('../saved_models/WaLeF_gtn_p.h5', custom_objects={'GCNConv': GCNConv})

ws_pred_gate_true = ws_predictor.predict([test_cov, test_tws_reshape, test_adj_mat])
ws_pred_gate_true_inv = ws_scaler.inverse_transform(ws_pred_gate_true)
ws_pred_gate_true_inv = ws_pred_gate_true_inv.reshape((-1, 24, 4))
ws_pred_gate_true_inv.shape

603/603 [==============================] - 3s 3ms/step


(19268, 24, 4)

### Upper threshould

In [19]:
upper_threshold = 3.5
t1 = 1

flood_threshold_t1(ws_true_gate_true_inv, t1, upper_threshold)
flood_threshold_t1(ws_pred_gate_true_inv, t1, upper_threshold)
flood_threshold_t1(ws_pred_gate_pred_inv, t1, upper_threshold)

S1, S25A, S25B, S26 time steps: 96, 96, 118, 117
S1, S25A, S25B, S26 areas: 14.82, 15.22, 18, 20.13
TOTAL time steps: 427; TOTAL areas: 68.61
--------------------------------------------------
S1, S25A, S25B, S26 time steps: 85, 85, 96, 108
S1, S25A, S25B, S26 areas: 11.5466, 12.1773, 13, 17.2181
TOTAL time steps: 374; TOTAL areas: 54.3063
--------------------------------------------------
S1, S25A, S25B, S26 time steps: 31, 52, 53, 45
S1, S25A, S25B, S26 areas: 3.7296, 6.8456, 8, 6.6839
TOTAL time steps: 181; TOTAL areas: 25.0683
--------------------------------------------------


In [20]:
flood_threshold_t1(ws_true_gate_true_inv[7640-23:7680-23], t1, upper_threshold)
flood_threshold_t1(ws_pred_gate_true_inv[7640-23:7680-23], t1, upper_threshold)
flood_threshold_t1(ws_pred_gate_pred_inv[7640-23:7680-23], t1, upper_threshold)

S1, S25A, S25B, S26 time steps: 6, 5, 6, 6
S1, S25A, S25B, S26 areas: 0.84, 0.72, 1, 1.0
TOTAL time steps: 23; TOTAL areas: 3.62
--------------------------------------------------
S1, S25A, S25B, S26 time steps: 6, 6, 5, 5
S1, S25A, S25B, S26 areas: 0.6148, 0.6425, 1, 0.8663
TOTAL time steps: 22; TOTAL areas: 3.0018
--------------------------------------------------
S1, S25A, S25B, S26 time steps: 2, 1, 1, 2
S1, S25A, S25B, S26 areas: 0.0625, 0.0478, 0, 0.14
TOTAL time steps: 6; TOTAL areas: 0.3257
--------------------------------------------------


In [21]:
flood_threshold(ws_true_gate_true_inv, upper_threshold)
flood_threshold(ws_pred_gate_true_inv, upper_threshold)
flood_threshold(ws_pred_gate_pred_inv, upper_threshold)

time steps: 10248, areas: 1646.6399002075195
time steps: 12130, areas: 2088.0380942821503
time steps: 8780, areas: 1458.1937358379364


### Lower threshold

In [22]:
lower_threshold = 0
t1 = 1

drought_threshold_t1(ws_true_gate_true_inv, t1, lower_threshold)
drought_threshold_t1(ws_pred_gate_true_inv, t1, lower_threshold)
drought_threshold_t1(ws_pred_gate_pred_inv, t1, lower_threshold)

S1, S25A, S25B, S26 time steps: 1346, 1341, 1229, 1250
S1, S25A, S25B, S26 areas: -385.8, -383.38, -345.08, -350.84:
TOTAL time steps: 5166; TOTAL areas: -1465.1
--------------------------------------------------
S1, S25A, S25B, S26 time steps: 1390, 1427, 1282, 1476
S1, S25A, S25B, S26 areas: -398.849, -392.0414, -350.2885, -429.5386:
TOTAL time steps: 5575; TOTAL areas: -1570.7176
--------------------------------------------------
S1, S25A, S25B, S26 time steps: 429, 302, 150, 228
S1, S25A, S25B, S26 areas: -84.3139, -61.0771, -22.0615, -40.0315:
TOTAL time steps: 1109; TOTAL areas: -207.484
--------------------------------------------------


In [23]:
lower_threshold = 0

drought_threshold(ws_true_gate_true_inv, lower_threshold)
drought_threshold(ws_pred_gate_true_inv, lower_threshold)
drought_threshold(ws_pred_gate_pred_inv, lower_threshold)

time steps: 124148, areas: 35182.6098122485
time steps: 127126, areas: 34960.07284716559
time steps: 25699, areas: 5103.557413006052
